# Problema 1
Identificar el genero a partir de un retrato de persona.

In [20]:
from PIL import Image
from IPython.display import display
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression



In [2]:
female_data = 'Female Faces'
male_data = 'Male Faces'

target_width = 128
target_height = 128

# data_dict = {
#    'features': [],
#    'labels': []
data_dict = {}

In [3]:
for i in os.listdir(female_data):
    if i.endswith('.jpg') or i.endswith('.png') or i.endswith('.jpeg'):  # Add more extensions if needed
        # Load the image
        image_path = os.path.join(female_data, i)
        image = Image.open(image_path).convert('RGB')
        resized_image = image.resize((target_width, target_height), Image.Resampling.LANCZOS)
        array=np.array(resized_image).flatten()
        array_tuple = tuple(array.tolist())
        # Append the resized image to the list
        data_dict[array_tuple] = (0)


/home/camilo/.local/share/virtualenvs/Algebra_Taller1_David_Camilo-pNXIVJ8t/lib/python3.11/site-packages/PIL/Image.py:970: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [4]:
for i in os.listdir(male_data):
    if i.endswith('.jpg') or i.endswith('.png') or i.endswith('.jpeg'):  # Add more extensions if needed
        # Load the image
        image_path = os.path.join(male_data, i)
        image = Image.open(image_path).convert('RGB')
        resized_image = image.resize((target_width, target_height),Image.Resampling.LANCZOS)
        array=np.array(resized_image).flatten()
        array_tuple = tuple(array.tolist())
        # Append the resized image to the list
        data_dict[array_tuple] = (1)

In [8]:
keys = list(data_dict.keys())
values = list(data_dict.values())

In [17]:
X = keys  # The image data (features)
y = values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
model = LogisticRegression()
model.fit(X_train, y_train)

/home/camilo/.local/share/virtualenvs/Algebra_Taller1_David_Camilo-pNXIVJ8t/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(report)

Accuracy: 0.7298136645962733
              precision    recall  f1-score   support

           0       0.76      0.70      0.73       165
           1       0.71      0.76      0.73       157

    accuracy                           0.73       322
   macro avg       0.73      0.73      0.73       322
weighted avg       0.73      0.73      0.73       322



In [22]:
input_image_path = 'Test_images/Male/1 (2360).jpg'
input_image = Image.open(input_image_path).convert('RGB')
resized_input_image = input_image.resize((target_width, target_height), Image.Resampling.LANCZOS)
input_array = np.array(resized_input_image).flatten()
input_tuple = tuple(input_array.tolist())


In [23]:
predicted_label = model.predict([input_tuple])

# Determine the result based on your label encoding (0 for female, 1 for male)
if predicted_label == 0:
    result = "Female"
else:
    result = "Male"

print(f"Predicted result: {result}")

Predicted result: Male


## Referencias

Dataset: https://www.kaggle.com/datasets/ashwingupta3012/male-and-female-faces-dataset